# SQL Analytics Project: Analyzing Product Sales in the Mobile App

## Problem

We have mobile app data of a large chain store. This data include information about installations, activity in the app (views) and purchases. App users are devided into two groups: those who use IOS platform and those who use Android platform. We need to compare installs and purchases of these groups and check if the are significant difference.

## Data

There are four tables, and their names and column definitions are listed below:

### Table 1: Installs

_This table includes information about app installations by day_

__DeviceID__ — a unique ID per device, on which the application was installed; 

__InstallationDate__ — date of app installation;

__InstallCost__ — price for installation;

__Platform__ — platform, on which the application was installed (iOS/ Android);

__Source__ — source of app installation (app store/ advertising system/ website traffic)

### Table 2: Events 

_This table includes information about user activity (views of products) by day_

__DeviceID__ — a unique ID per device, on which the application was installed; 

__AppPlatform__ — platform, on which the application is used (iOS/ Android);

__EventDate__ — date of statistics collection;

__events__ — the number of views of all products for this day at this DeviceID.

The feature of the application is that user does not need authorization to view products. Until the moment of authorization the user has only DeviceID (the device identifier). A login is required only to make a purchase. At the moment of authorization, the user is assigned a UserID.

### Table 3: Checks

_This table includes information about purchases in the app by day_

__UserID__ — a unique ID per user;

__Rub__ — the user's total receipt for the date;

__BuyDate__ — date of statistics collection.

### Table 4: Devices

__DeviceID__ — a unique ID per device;

__UserID__ — a unique ID per user.

## Analysis

Importing libraries

In [2]:
import pandahouse as ph
connection = dict(database='default',
                  host='https://clickhouse.lab.karpov.courses',
                  user='student',
                  password='dpo_python_2020')

First of all we need to choose metrics that help us compare two platforms.

I will use conversion from viewing to autorization, from autorization to purchase, average check and average views

In [7]:
query = """     
SELECT 
    ev.AppPlatform,
    count(DISTINCT DeviceID) AS events_q, --the number users who have views
    a.autoriz_q,
    a.autoriz_q/events_q as CR --convertion from view to autorization
FROM 
    events ev
JOIN 
    (SELECT
        e.AppPlatform,
        count(DISTINCT DeviceID) AS autoriz_q --the number of authorized users who have views
    FROM 
        events e
    JOIN 
        devices d
    USING
        (DeviceID)
    GROUP BY 
    e.AppPlatform) AS a
ON 
    a.AppPlatform = ev.AppPlatform
GROUP BY 
    AppPlatform, a.autoriz_q
"""
df = ph.read_clickhouse(query, connection=connection)
df

,AppPlatform,events_q,autoriz_q,CR
0,iOS,4298120,2284988,0.531625
1,android,14110738,7121648,0.504697


In [4]:
query = """
SELECT 
    Platform, 
    autoriz_q, 
    purch_q, 
    purch_q/autoriz_q AS CR --covertion from utorization to purchase
FROM
    (SELECT 
        in.Platform, 
        count(DISTINCT dv.DeviceID) AS autoriz_q --the number of authorized users
    FROM 
        devices dv 
    JOIN 
        installs in
    ON 
        dv.DeviceID = in.DeviceID
    GROUP BY 
        in.Platform) AS a
JOIN
    (SELECT 
        i.Platform, 
        count(DISTINCT d.DeviceID) AS purch_q --the number of authorized users who made the purchases
    FROM 
        devices d 
    JOIN 
        installs i
    ON 
        d.DeviceID = i.DeviceID
    JOIN 
        checks c
    ON 
        c.UserID=d.UserID
    GROUP BY 
        i.Platform) AS p
ON 
    i.Platform = a.Platform
GROUP BY 
    Platform, autoriz_q, purch_q
"""
df = ph.read_clickhouse(query, connection=connection)
df

,Platform,autoriz_q,purch_q,CR
0,android,4625795,3027287,0.654436
1,iOS,1105145,564247,0.510564


In [3]:
query = """
SELECT 
    i.Platform, avg(c.Rub) AS avg_sum --average check
FROM 
    checks c
JOIN 
    devices d
ON 
    c.UserID = d.UserID
JOIN 
    installs i
ON 
    i.DeviceID = d.DeviceID
GROUP BY 
    i.Platform
"""
df = ph.read_clickhouse(query, connection=connection)
df

,i.Platform,avg_sum
0,android,10.394975
1,iOS,11.024963


In [5]:
query = """
SELECT 
    AppPlatform, 
    avg(e.events) AS avg_event --average views
FROM 
    events e
GROUP BY 
    AppPlatform
"""
df = ph.read_clickhouse(query, connection=connection)
df

,AppPlatform,avg_event
0,iOS,25.424526
1,android,26.006177


## Conclusion

We can see that the conversion from viewing to authorization is slightly higher for iOS users, and the conversion from authorization to purchase is higher for Android users. This may indicate that the purchase page is probably worse configured in the iOS app, which stops users from buying after registration. At the same time, the average check and average views are practically the same on the two platforms.